# HOMEWORK 8 Neural Network and Deep Learning

In [2]:
%pip install torch

<frozen importlib._bootstrap>:488: Warning: OpenSSL 3's legacy provider failed to load. Legacy algorithms will not be available. If you need those algorithms, check your OpenSSL configuration.
<frozen importlib._bootstrap>:488: Warning: OpenSSL 3's legacy provider failed to load. Legacy algorithms will not be available. If you need those algorithms, check your OpenSSL configuration.
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install torchvision

<frozen importlib._bootstrap>:488: Warning: OpenSSL 3's legacy provider failed to load. Legacy algorithms will not be available. If you need those algorithms, check your OpenSSL configuration.
<frozen importlib._bootstrap>:488: Warning: OpenSSL 3's legacy provider failed to load. Legacy algorithms will not be available. If you need those algorithms, check your OpenSSL configuration.
Note: you may need to restart the kernel to use updated packages.


## Dataset

In [1]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip


--2025-12-02 06:51:22--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-02T07%3A30%3A14Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-02T06%3A29%3A28Z&ske=2025-12-02T07%3A30%3A14Z&sks=b&skv=2018-11-09&sig=HkbHTvMARYIKs8d5ILInnFzP28rRmCs8vvsHwOLdWGE%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDY2MDA4MiwibmJmIjoxNzY0NjU4MjgyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG

## Reproducibility

In [2]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Model

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3), padding=0, stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 2))
        )
        self.classifier = nn.Sequential(
            nn.Linear(32 * 99 * 99, 64), 
            nn.ReLU(),
            nn.Linear(64, 1) 

        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

model = CNN()
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)



## Question 1

Which loss function you will use?

* nn.MSELoss()
* `nn.BCEWithLogitsLoss()`
* nn.CrossEntropyLoss()
* nn.CosineEmbeddingLoss()

In [4]:
import torch.nn as nn


criterion = nn.BCEWithLogitsLoss()


## Question 2

What's the total number of parameters of the model? You can use `torchsummary` or count manually. 

* 896 
* 11214912
* 15896912
* `20073473`


In [5]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")


Total parameters: 20073473


## Generators and Training

In [6]:
from torchvision import datasets, transforms


train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [7]:
val_transforms = transforms.Compose([
    transforms.Resize((200, 200)), # Resize all images to a consistent size, e.g., 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


            


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torchvision.transforms as T
batch_size = 20

train_dir = "data/train"
test_dir = "data/test"

class_mode = "categorical"
# Define a common target size for all images
# For example, resize all images to 640x640
target_size = (200, 200) 
resize_transform = T.Resize(target_size)

# Create a resize transform
train_datagen = ImageDataGenerator() # No additional preprocessing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    shuffle=True
)

test_datagen = ImageDataGenerator() # No additional preprocessing
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode=class_mode,
    shuffle=False
)


Found 800 images belonging to 2 classes.
Found 201 images belonging to 2 classes.


In [10]:
from torch.utils.data import DataLoader, TensorDataset

from torchvision import datasets, transforms

train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
validation_dataset = datasets.ImageFolder(test_dir, transform = val_transforms)

# Define the batch size
batch_size = 64 # Or any other appropriate batch size

# Create the DataLoader for the training data
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)



In [11]:

# if torch.cuda.is_available():
#     device = torch.device("cuda")
#     print(f"CUDA is available! Using GPU: {torch.cuda.get_device_name(0)}")
# else:
#     device = torch.device("cpu")
#     print("CUDA is not available. Using CPU instead.")

device = torch.device("cpu")

# You can now use 'device' in your model and tensor operations, e.g.:
# model.to(device)
# data.to(device)


In [ ]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6532, Acc: 0.6262, Val Loss: 0.6207, Val Acc: 0.6716
Epoch 2/10, Loss: 0.5471, Acc: 0.7100, Val Loss: 0.9127, Val Acc: 0.5871
Epoch 3/10, Loss: 0.5899, Acc: 0.7025, Val Loss: 0.5991, Val Acc: 0.6617
Epoch 4/10, Loss: 0.4520, Acc: 0.7950, Val Loss: 0.6652, Val Acc: 0.6318
Epoch 5/10, Loss: 0.4402, Acc: 0.7913, Val Loss: 0.6098, Val Acc: 0.6517
Epoch 6/10, Loss: 0.3723, Acc: 0.8350, Val Loss: 0.6258, Val Acc: 0.6816
Epoch 7/10, Loss: 0.3117, Acc: 0.8775, Val Loss: 0.7521, Val Acc: 0.6368
Epoch 8/10, Loss: 0.2915, Acc: 0.8800, Val Loss: 0.6390, Val Acc: 0.7164
Epoch 9/10, Loss: 0.2464, Acc: 0.9200, Val Loss: 0.8028, Val Acc: 0.6617
Epoch 10/10, Loss: 0.2825, Acc: 0.8712, Val Loss: 0.6741, Val Acc: 0.6716


## Question 3

What is the median of training accuracy for all the epochs for this model?

* 0.05
* 0.12
* 0.40
* `0.84`

In [ ]:
np.median(history['acc'])


np.float64(0.815)

## Question 4

What is the standard deviation of training loss for all the epochs for this model?

* 0.007
* 0.078
* `0.171`
* 1.710

In [ ]:



training_losses = history['loss']
std_dev_training_loss = np.std(training_losses)
print(f"Standard deviation of training loss: {std_dev_training_loss:.4f}")



Standard deviation of training loss: 0.1343


## Data Augmentation

In [ ]:
transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip(),

(RandomHorizontalFlip(p=0.5),)

## Question 5

Let's train our model for 10 more epochs using the same code as previously.

> **Note:** make sure you don't re-create the model.
> we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.008
* 0.08
* 0.88
* 8.88

In [ ]:
num_epochs = 20
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/20, Loss: 0.1910, Acc: 0.9487, Val Loss: 0.6531, Val Acc: 0.7164
Epoch 2/20, Loss: 0.1665, Acc: 0.9437, Val Loss: 0.6738, Val Acc: 0.6965
Epoch 3/20, Loss: 0.1246, Acc: 0.9775, Val Loss: 0.7079, Val Acc: 0.7114
Epoch 4/20, Loss: 0.0942, Acc: 0.9912, Val Loss: 0.7305, Val Acc: 0.7313
Epoch 5/20, Loss: 0.1087, Acc: 0.9750, Val Loss: 0.7523, Val Acc: 0.7015
Epoch 6/20, Loss: 0.0677, Acc: 0.9950, Val Loss: 0.7691, Val Acc: 0.7313
Epoch 7/20, Loss: 0.0497, Acc: 1.0000, Val Loss: 0.7516, Val Acc: 0.7214
Epoch 8/20, Loss: 0.0472, Acc: 0.9975, Val Loss: 0.8698, Val Acc: 0.6965
Epoch 9/20, Loss: 0.0375, Acc: 1.0000, Val Loss: 0.7999, Val Acc: 0.7313
Epoch 10/20, Loss: 0.0409, Acc: 1.0000, Val Loss: 0.8134, Val Acc: 0.7413
Epoch 11/20, Loss: 0.0263, Acc: 1.0000, Val Loss: 0.8756, Val Acc: 0.7413
Epoch 12/20, Loss: 0.0210, Acc: 1.0000, Val Loss: 0.8967, Val Acc: 0.7214
Epoch 13/20, Loss: 0.0189, Acc: 1.0000, Val Loss: 0.8697, Val Acc: 0.7413
Epoch 14/20, Loss: 0.0160, Acc: 1.0000, Val Los

## Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

* 0.08
* 0.28
* `0.68`
* 0.98

In [ ]:


average_test_accuracy = np.mean(history['val_acc'][5:10])
print(average_test_accuracy)



0.7243781094527363
